In [112]:
import pandas as pd
import torch
import sys

In [113]:
# obtaining data
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)

X = torch.Tensor([df.TV.to_numpy()]).transpose(0, 1)
y = torch.Tensor([df.Sales.to_numpy()]).transpose(0, 1)

In [114]:
X.shape

torch.Size([200, 1])

In [115]:
y.shape

torch.Size([200, 1])

In [116]:
# prior parameters
a0 = 1e-2
b0 = 1e-4
c0 = 1e-2
d0 = 1e-4

In [117]:
# pre-process data
N = X.shape[0]
D = 1
if len(X.shape) > 1:
    D = X.shape[1]

X_corr = X.transpose(0, 1).matmul(X)
Xy_corr = X.transpose(0, 1).matmul(y)

an = a0 + N / 2
gammaln_an = torch.lgamma(torch.Tensor([an]))
cn = c0 + D / 2
gammaln_cn = torch.lgamma(torch.Tensor([cn]))

In [118]:
# iterate to find hyperparameters
L_last = -sys.float_info.max
max_iter = 500
E_a = c0 / d0

for i in range(max_iter):
    # covariance and weight of linear model
    invV = E_a * torch.eye(D) + X_corr
    V = torch.inverse(invV)
    logdetV = - torch.logdet(invV)
    w = V.matmul(Xy_corr)

    # parameters of noise model (an remains constant)
    sse = torch.sum((X.matmul(w) - y) ** 2)
    bn = b0 + 0.5 * (sse + E_a * w.transpose(0, 1).matmul(w))
    E_t = an / bn

    # hyperparameters of covariance prior (cn remains constant)
    dn = d0 + 0.5 * (E_t * w.transpose(0, 1).matmul(w) + torch.trace(V))
    E_a = cn / dn

    # variational bound, ignoring constant terms for now
    L = -0.5 * (E_t * sse + torch.sum(torch.sum(X * (X.matmul(V))))) + 0.5 * logdetV \
        - b0 * E_t + gammaln_an - an * torch.log(bn) + an \
        + gammaln_cn - cn * torch.log(dn)
    
    print(L)
    
    # variational bound must grow!
    if L_last > L:
        print('Last bound:', L_last)
        print('Current bound:', L)
        print('Variational bound should not reduce')
        break

    # stop if change in variation bound is < 0.001%
    if abs(L_last - L) < abs(0.00001 * L):
        break
    L_last = L

    if iter == max_iter:
        print('Bayesian linear regression reached maximum number of iterations')

    # augment variational bound with constant terms
    L = L - 0.5 * (N * torch.log(torch.Tensor([2 * torch.pi])) - D) - torch.lgamma(torch.Tensor([a0])) + a0 * torch.log(torch.Tensor([b0])) \
        - torch.lgamma(torch.Tensor([c0])) + c0 * torch.log(torch.Tensor([d0]))

tensor([[-418.6396]])
tensor([[-418.6392]])


In [119]:
print("InvV:", invV)
print("V:", V)
print("logdetV:", logdetV)
print("w", w)
print("sse:", sse)
print("bn:", bn)
print("E_t:", E_t)
print("dn:", dn)
print("E_a:", E_a)
print("an:", an)
print("gammaln_an:", gammaln_an)
print("cn:", cn)
print("gammaln_cn:", gammaln_cn)

InvV: tensor([[5793154.]])
V: tensor([[1.7262e-07]])
logdetV: tensor(-15.5722)
w tensor([[0.0832]])
sse: tensor(4607.9243)
bn: tensor([[2311.0129]])
E_t: tensor([[0.0433]])
dn: tensor([[0.0002]])
E_a: tensor([[2040.4816]])
an: 100.01
gammaln_an: tensor([359.1802])
cn: 0.51
gammaln_cn: tensor([0.5530])
